# Protein -participates in-> Reaction
These proteins participate in Reactome reactions. Only uses the human reactions

In [2]:
import pandas as pd
import numpy as np
import json
import os
from biomed_apis import *
from biomedkg_utils import *

In [3]:
os.system('wget -N -P input/ https://reactome.org/download/current/ProteinRoleReaction.txt')

--2023-03-08 23:41:55--  https://reactome.org/download/current/ProteinRoleReaction.txt
Resolving reactome.org (reactome.org)... 100.25.71.177
Connecting to reactome.org (reactome.org)|100.25.71.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39692424 (38M) [text/plain]
Saving to: ‘input/ProteinRoleReaction.txt’

     0K .......... .......... .......... .......... ..........  0%  398K 97s
    50K .......... .......... .......... .......... ..........  0%  779K 73s
   100K .......... .......... .......... .......... ..........  0%  779K 65s
   150K .......... .......... .......... .......... ..........  0%  390K 74s
   200K .......... .......... .......... .......... ..........  0%  778K 69s
   250K .......... .......... .......... .......... ..........  0%  780K 65s
   300K .......... .......... .......... .......... ..........  0%  779K 63s
   350K .......... .......... .......... .......... ..........  1%  779K 61s
   400K .......... .......... ..........

  5800K .......... .......... .......... .......... .......... 15%  903K 13s
  5850K .......... .......... .......... .......... .......... 15%  163M 13s
  5900K .......... .......... .......... .......... .......... 15%  185M 13s
  5950K .......... .......... .......... .......... .......... 15%  162M 13s
  6000K .......... .......... .......... .......... .......... 15%  130M 12s
  6050K .......... .......... .......... .......... .......... 15%  169M 12s
  6100K .......... .......... .......... .......... .......... 15%  171M 12s
  6150K .......... .......... .......... .......... .......... 15%  171M 12s
  6200K .......... .......... .......... .......... .......... 16%  118M 12s
  6250K .......... .......... .......... .......... .......... 16%  151M 12s
  6300K .......... .......... .......... .......... .......... 16%  147M 12s
  6350K .......... .......... .......... .......... .......... 16%  268M 12s
  6400K .......... .......... .......... .......... .......... 16%  233M 12s

 16500K .......... .......... .......... .......... .......... 42%  998K 4s
 16550K .......... .......... .......... .......... .......... 42% 88.3M 4s
 16600K .......... .......... .......... .......... .......... 42%  236M 4s
 16650K .......... .......... .......... .......... .......... 43%  255M 4s
 16700K .......... .......... .......... .......... .......... 43%  259M 4s
 16750K .......... .......... .......... .......... .......... 43%  285M 4s
 16800K .......... .......... .......... .......... .......... 43% 5.96M 4s
 16850K .......... .......... .......... .......... .......... 43%  268M 4s
 16900K .......... .......... .......... .......... .......... 43%  236M 4s
 16950K .......... .......... .......... .......... .......... 43%  272M 4s
 17000K .......... .......... .......... .......... .......... 43%  226M 4s
 17050K .......... .......... .......... .......... .......... 44%  297M 4s
 17100K .......... .......... .......... .......... .......... 44%  260M 3s
 17150K ....

 26100K .......... .......... .......... .......... .......... 67%  196M 2s
 26150K .......... .......... .......... .......... .......... 67%  240M 2s
 26200K .......... .......... .......... .......... .......... 67%  259M 2s
 26250K .......... .......... .......... .......... .......... 67%  271M 1s
 26300K .......... .......... .......... .......... .......... 67%  280M 1s
 26350K .......... .......... .......... .......... .......... 68%  283M 1s
 26400K .......... .......... .......... .......... .......... 68%  228M 1s
 26450K .......... .......... .......... .......... .......... 68%  181M 1s
 26500K .......... .......... .......... .......... .......... 68%  168M 1s
 26550K .......... .......... .......... .......... .......... 68%  160M 1s
 26600K .......... .......... .......... .......... .......... 68%  153M 1s
 26650K .......... .......... .......... .......... .......... 68%  162M 1s
 26700K .......... .......... .......... .......... .......... 69%  167M 1s
 26750K ....

 31900K .......... .......... .......... .......... .......... 82% 1.09M 1s
 31950K .......... .......... .......... .......... .......... 82%  251M 1s
 32000K .......... .......... .......... .......... .......... 82%  236M 1s
 32050K .......... .......... .......... .......... .......... 82% 7.42M 1s
 32100K .......... .......... .......... .......... .......... 82% 71.5M 1s
 32150K .......... .......... .......... .......... .......... 83% 7.08M 1s
 32200K .......... .......... .......... .......... .......... 83%  192M 1s
 32250K .......... .......... .......... .......... .......... 83% 92.1M 1s
 32300K .......... .......... .......... .......... .......... 83%  211M 1s
 32350K .......... .......... .......... .......... .......... 83%  282M 1s
 32400K .......... .......... .......... .......... .......... 83%  220M 1s
 32450K .......... .......... .......... .......... .......... 83%  287M 1s
 32500K .......... .......... .......... .......... .......... 83%  235M 1s
 32550K ....

0

## Proteins

In [4]:
'''Get proteins'''
protein2reaction, reaction2protein = dict(), dict()
proteins = set()

for line in open('input/ProteinRoleReaction.txt'):
    line = line.split('\t')
    
    # Protein, reaction, reaction type
    protein = line[0]
    react_type = line[1]
    reaction = line[2].strip()
    
    # Human reaction?
    human_reaction = reaction.startswith('R-HSA')
    if human_reaction:
        proteins.add(protein)

In [5]:
''' Filter for reviewed human proteins'''
job_id = submit_id_mapping_UniProtAPI(
                  from_db = 'UniProtKB_AC-ID',
                  to_db = 'UniProtKB-Swiss-Prot', 
                  ids = list(proteins))

# This checks on the job until it is finished
if check_id_mapping_results_ready_UniProtAPI(job_id):
    link = get_id_mapping_results_link_UniProtAPI(job_id)
    results = get_id_mapping_results_search_UniProtAPI(link)
    

human_proteins = set()
for i in range(0,len(results['results'])):
    res = results['results'][i]
    protein = res['from']

    try:
        reviewed = 'reviewed' in res['to']['entryType']
        human = res['to']['organism']['commonName'] == 'Human'
    except:
        continue
        
    if reviewed and human:
        human_proteins.add(protein)

Job still running. Retrying in 3s
Job still running. Retrying in 3s
Job still running. Retrying in 3s
Job still running. Retrying in 3s
Job still running. Retrying in 3s


### Protein - Reaction

In [31]:
protein2reaction, reaction2protein = dict(), dict()
proteins = set()

file = 'Protein_(UniProt)_2_Reaction_(Reactome).csv'
with open(os.path.join('output/protein2reaction',file),'w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Protein (UniProt)','Reaction (Reactome)', 'Relationship'])
    
    for line in open('input/ProteinRoleReaction.txt'):
        line = line.split('\t')

        # Protein, reaction, reaction type
        protein = line[0]
        react_type = line[1]
        reaction = line[2].strip()

        # Human reaction?
        human_reaction = reaction.startswith('R-HSA')
        if human_reaction:

            # Human protein?
            if protein in human_proteins:

                # Protein - Reaction
                protein2reaction.setdefault(protein, set()).add(reaction)
                reaction2protein.setdefault(reaction, set()).add(protein)
                writer.writerow(['UniProt:'+protein, 'Reactome_Reaction:'+reaction, '-'+react_type+'->'])

df = pd.read_csv(os.path.join('output/protein2reaction',file)).drop_duplicates()
df.to_csv(os.path.join('output/edges',file),index=False)
df.to_csv(os.path.join('output/edges to use',file),index=False)

print(len(protein2reaction), 'Proteins')
print(len(reaction2protein), 'Reactions')

10938 Proteins
13267 Reactions


## Compound -particiaptes in-> Reaction

In [6]:
os.system('wget -N -P input/ https://reactome.org/download/current/ChEBI2Reactome_PE_Reactions.txt')

--2023-03-08 23:45:29--  https://reactome.org/download/current/ChEBI2Reactome_PE_Reactions.txt
Resolving reactome.org (reactome.org)... 100.25.71.177
Connecting to reactome.org (reactome.org)|100.25.71.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38975444 (37M) [text/plain]
Saving to: ‘input/ChEBI2Reactome_PE_Reactions.txt’

ChEBI2Reactome_PE_R 100%[===================>]  37.17M  15.4MB/s    in 2.4s    

2023-03-08 23:45:32 (15.4 MB/s) - ‘input/ChEBI2Reactome_PE_Reactions.txt’ saved [38975444/38975444]



In [23]:
'''Get compounds'''
chebi2db = json.load(open('output/compound2compound/chebi2db.json'))
chebi2mesh = json.load(open('output/compound2compound/chebi2mesh.json'))

mesh_compound2reactome_reaction, reactome_reaction2mesh_compound = dict(), dict()
db_compound2reactome_reaction, reactome_reaction2db_compound = dict(), dict()
chebi2reaction, reaction2chebi = dict(), dict()

compounds = set()

for line in open('input/ChEBI2Reactome_PE_Reactions.txt'):
    line = line.split('\t')

    # Compound, Reaction
    chebi = line[0]
    reaction = line[3]
    
    # Human reaction?
    human_reaction = reaction.startswith('R-HSA')
    if human_reaction:
        
        # ChEBI Compound
        chebi2reaction.setdefault(chebi, set()).add(reaction)
        reaction2chebi.setdefault(reaction, set()).add(chebi)
        
        
        # MeSH Compound
        try:
            mesh_compounds = chebi2mesh[chebi]
            for mesh_compound in mesh_compounds:
                mesh_compound2reactome_reaction.setdefault(mesh_compound, set()).add(reaction)
                reactome_reaction2mesh_compound.setdefault(reaction, set()).add(mesh_compound)
                
        except:
            pass
        
        
        # DrugBank Compound
        try:
            db_compounds = chebi2db[chebi]
            for db_compound in db_compounds:
                db_compound2reactome_reaction.setdefault(db_compound, set()).add(reaction)
                reactome_reaction2db_compound.setdefault(reaction, set()).add(db_compound)
                
        except:
            pass
        
print('DrugBank Compounds:', len(db_compound2reactome_reaction), 
      '\nMeSH Compounds:', len(mesh_compound2reactome_reaction), 
      '\nChEBI Compounds:', len(chebi2reaction))
print('Reactions to MeSH Compounds:', len(reactome_reaction2mesh_compound), 
      '\nReactions to DrugBank Drugs',len(reactome_reaction2db_compound), 
      '\nReactions to ChEBI:',len(reaction2chebi))

DrugBank Compounds: 582 
MeSH Compounds: 589 
ChEBI Compounds: 2391
Reactions to MeSH Compounds: 2957 
Reactions to DrugBank Drugs 3709 
Reactions to ChEBI: 8673


In [27]:
file = 'Compound_(DrugBank)_2_Reaction_(Reactome).csv'
outpath = os.path.join('output/compound2reaction',file)
output_edgefile_onerel_noweight(outpath,
                               ['Compound (DrugBank)', 'Reaction (Reactome)', 'Relationship'],
                               db_compound2reactome_reaction,
                                '-participates_in->',
                               'DrugBank_Compound:',
                               'Reactome_Reaction:')
df = pd.read_csv(os.path.join('output/compound2reaction',file)).drop_duplicates()
df.to_csv(os.path.join('output/edges',file), index=False)
df.to_csv(os.path.join('output/edges to use',file), index=False)

In [28]:
df

,Compound (DrugBank),Reaction (Reactome),Relationship
0,DrugBank_Compound:DB13781,Reactome_Reaction:R-HSA-9611721,-participates_in->
1,DrugBank_Compound:DB13781,Reactome_Reaction:R-HSA-390674,-participates_in->
2,DrugBank_Compound:DB01392,Reactome_Reaction:R-HSA-9728747,-participates_in->
3,DrugBank_Compound:DB01392,Reactome_Reaction:R-HSA-9728752,-participates_in->
4,DrugBank_Compound:DB01392,Reactome_Reaction:R-HSA-749456,-participates_in->
...,...,...,...
6655,DrugBank_Compound:DB00208,Reactome_Reaction:R-HSA-417829,-participates_in->
6656,DrugBank_Compound:DB00839,Reactome_Reaction:R-HSA-9652580,-participates_in->
6657,DrugBank_Compound:DB00519,Reactome_Reaction:R-HSA-9619024,-participates_in->
6658,DrugBank_Compound:DB00177,Reactome_Reaction:R-HSA-9615249,-participates_in->


In [29]:
file = 'Compound_(MeSH)_2_Reaction_(Reactome).csv'
outpath = os.path.join('output/compound2reaction',file)
output_edgefile_onerel_noweight(outpath,
                               ['Compound (MeSH)', 'Reaction (Reactome)', 'Relationship'],
                               mesh_compound2reactome_reaction,
                                '-participates_in->',
                               'MeSH_Compound:',
                               'Reactome_Reaction:')
df = pd.read_csv(os.path.join('output/compound2reaction',file)).drop_duplicates()
df.to_csv(os.path.join('output/edges',file), index=False)
df.to_csv(os.path.join('output/edges to use',file), index=False)

In [30]:
df

,Compound (MeSH),Reaction (Reactome),Relationship
0,MeSH_Compound:D017307,Reactome_Reaction:R-HSA-9611721,-participates_in->
1,MeSH_Compound:D017307,Reactome_Reaction:R-HSA-390674,-participates_in->
2,MeSH_Compound:D015016,Reactome_Reaction:R-HSA-9728747,-participates_in->
3,MeSH_Compound:D015016,Reactome_Reaction:R-HSA-9728752,-participates_in->
4,MeSH_Compound:D015016,Reactome_Reaction:R-HSA-749456,-participates_in->
...,...,...,...
5269,MeSH_Compound:C052035,Reactome_Reaction:R-HSA-9619024,-participates_in->
5270,MeSH_Compound:D000068756,Reactome_Reaction:R-HSA-9615249,-participates_in->
5271,MeSH_Compound:D000068756,Reactome_Reaction:R-HSA-374173,-participates_in->
5272,MeSH_Compound:C081489,Reactome_Reaction:R-HSA-9615249,-participates_in->
